<a href="https://colab.research.google.com/github/2303a51lb8/Sai-Ganesh-NLP/blob/main/NLP_ASSIGNMENT_LAB_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Use a small dataset of English to French sentence pairs. You can replace this with any other language pair dataset.
data = [ ("hello", "bonjour"), ("how are you", "comment ça va"), ("I am fine", "je vais bien"), ("what is your name", "comment tu t'appelles"), ("my name is", "je m'appelle"), ("thank you", "merci"), ("goodbye", "au revoir") ]

(a) Take input (English) and target (French)

(b) Building the Seq2Seq Model

(c) Prepare the target sequences for training by shifting them by one position, which the model will use to predict the next word in the sequence

(d) After training, set up separate models for the encoder and decoder to perform inference (translation) on new sentences.

(e) Use the trained model to translate new English sentences into French

In [4]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = [
    ("hello", "bonjour"),
    ("thanks", "merci"),
    ("yes", "oui"),
    ("no", "non"),
    ("please", "s'il vous plaît"),
    ("sorry", "désolé"),
    ("goodbye", "au revoir")
]

START_TOKEN = '<start>'
END_TOKEN = '<end>'
PAD_TOKEN = '<pad>'

input_texts = [pair[0] for pair in data]
target_texts = [f'{START_TOKEN} {pair[1]} {END_TOKEN}' for pair in data]

input_tokenizer = Tokenizer(filters='')
target_tokenizer = Tokenizer(filters='')

input_tokenizer.fit_on_texts([PAD_TOKEN] + input_texts)
target_tokenizer.fit_on_texts([PAD_TOKEN, START_TOKEN, END_TOKEN] + target_texts)

# Convert text to sequences
input_sequences = input_tokenizer.texts_to_sequences(input_texts)
target_sequences = target_tokenizer.texts_to_sequences(target_texts)

max_input_length = max(len(seq) for seq in input_sequences)
max_target_length = max(len(seq) for seq in target_sequences)

input_sequences = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')
target_sequences = pad_sequences(target_sequences, maxlen=max_target_length, padding='post')

# Model parameters
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1
embedding_dim = 128
lstm_units = 256

# Create training data
decoder_input_data = target_sequences[:, :-1]
decoder_target_data = target_sequences[:, 1:]
decoder_target_data = np.expand_dims(decoder_target_data, -1)  # Reshape to (batch_size, seq_length, 1)

# Encoder
encoder_inputs = Input(shape=(max_input_length,))
encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(target_vocab_size, embedding_dim)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_dense = Dense(target_vocab_size, activation='softmax')

# Connect decoder to encoder
decoder_embedded = decoder_embedding(decoder_inputs)
decoder_outputs, _, _ = decoder_lstm(decoder_embedded, initial_state=encoder_states)
decoder_outputs = decoder_dense(decoder_outputs)

# Create the training model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(
    [input_sequences, decoder_input_data],
    decoder_target_data,
    batch_size=2,
    epochs=500,
    validation_split=0.2,
    verbose=1
)

# Encoder model
encoder_model_inf = Model(encoder_inputs, encoder_states)

# Decoder model
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedded_inf = decoder_embedding(decoder_inputs)
decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(
    decoder_embedded_inf, initial_state=decoder_states_inputs)
decoder_states_inf = [state_h_inf, state_c_inf]
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)

decoder_model_inf = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs_inf] + decoder_states_inf)

def translate_sentence(input_sentence):
    input_seq = input_tokenizer.texts_to_sequences([input_sentence])
    input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')

    states_value = encoder_model_inf.predict(input_seq, verbose=0)

    target_seq = np.array([[target_tokenizer.word_index[START_TOKEN]]])

    # Initialize variables
    decoded_sentence = []
    stop_condition = False

    while not stop_condition:
        output_tokens, h, c = decoder_model_inf.predict([target_seq] + states_value, verbose=0)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = target_tokenizer.index_word.get(sampled_token_index, '')

        if sampled_word == END_TOKEN or len(decoded_sentence) > max_target_length:
            stop_condition = True
        elif sampled_word != START_TOKEN:
            decoded_sentence.append(sampled_word)

        target_seq = np.array([[sampled_token_index]])

        states_value = [h, c]

    return ' '.join(decoded_sentence)

# Test the model
test_sentences = [
    "hello",
    "thanks",
    "goodbye"
]

print("\nTesting translations:")
for sentence in test_sentences:
    translation = translate_sentence(sentence)
    print(f'Input: {sentence}')
    print(f'Translation: {translation}\n')


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 1, 128)         │          1,152 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, None, 128)      │          1,792 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 256), (None,   │        394,240 │ embedding[0][0]        │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, None, 256),    │        394,240 │ embedding_1[0][0],     │
│                           │ (None, 256), (None,    │                │ lstm_2[0][1],          │
│                           │ 256)]                  │                │ lstm_2[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, None, 14)       │          3,598 │ lstm_3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 795,022 (3.03 MB)

 Trainable params: 795,022 (3.03 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 427ms/step - accuracy: 0.1969 - loss: 2.6349 - val_accuracy: 0.5000 - val_loss: 2.6064
Epoch 2/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4281 - loss: 2.5814 - val_accuracy: 0.3750 - val_loss: 2.5617
Epoch 3/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.3562 - loss: 2.5125 - val_accuracy: 0.3750 - val_loss: 2.4914
Epoch 4/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.3562 - loss: 2.4055 - val_accuracy: 0.3750 - val_loss: 2.3719
Epoch 5/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.3562 - loss: 2.2206 - val_accuracy: 0.3750 - val_loss: 2.1684
Epoch 6/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.4500 - loss: 1.8359 - val_accuracy: 0.3750 - val_loss: 1.8811
Epoch 7/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.4187 - loss: 1.5438 - val_accuracy: 0.3750 - val_loss: 1.6818
Epoch 8/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.4187 - loss: 1.4051 - val_accuracy: 0.3750 - val_loss